In [34]:
# import python libraries
import os
import re
from datetime import datetime
from glob import glob
from fnmatch import fnmatch
import pandas as pd
from docx import Document
import slate3k as slate
import openpyxl
import csv

In [2]:
# put cautions at the start of script execution
print("Things to know before proceeding:")
print("1. You must be connected to the BCGOV network by VPN or ethernet cable.")
print("2. You need to have OneDrive set up because your search results will be saved there as a CSV file.")
print("3. This script uses REGEX to find specific content within files. If you are adding new keywords to the search, you may want to visit https://www.dataquest.io/blog/regex-cheatsheet/.")
print("\n")

Things to know before proceeding:
1. You must be connected to the BCGOV network by VPN or ethernet cable.
2. You need to have OneDrive set up because your search results will be saved there as a CSV file.
3. This script uses REGEX to find specific content within files. If you are adding new keywords to the search, you may want to visit https://www.dataquest.io/blog/regex-cheatsheet/.




In [3]:
# set the date formatting
now = datetime.now()
dt_string = now.strftime("%Y-%b-%d-%H%M%S")

In [11]:
# set the file paths
UserName = os.environ.get('USERNAME')

OneDrive = f"C:/Users/{UserName}/OneDrive - Government of BC"
if not os.path.exists(OneDrive):
  print("Please enable OneDrive before running this script")
  exit()

SavePath = f"C:/Users/{UserName}/OneDrive - Government of BC/Optimize"
if not os.path.exists(SavePath):
  os.makedirs(SavePath)

SaveName = f"{SavePath}/filtered_path_report_" + dt_string + ".csv"

In [5]:
keywords = ["^first aid$|^First aid$|^First Aid$","[6][A]|[6][a]","SIRP|sirp",r"\btrauma|\bTrauma",r"\bbully|\bBully",r"\bbullied|\Bullied",r"\bharass|\bHarass",r"\bsuicid|\bSuicid",r"\bassault|\bAssault",r"\babus|\bAbus"]

print(f"The keyword list is: \n")
for (i, item) in enumerate(keywords, start=0):
    print(i, item)

The keyword list is: 

0 ^first aid$|^First aid$|^First Aid$
1 [6][A]|[6][a]
2 SIRP|sirp
3 \btrauma|\bTrauma
4 \bbully|\bBully
5 \bbullied|\Bullied
6 \bharass|\bHarass
7 \bsuicid|\bSuicid
8 \bassault|\bAssault
9 \babus|\bAbus


In [6]:
keywords_answer = input("Do you want to add more keywords to the existing list? ")

if keywords_answer == "yes":
  # add new keywords to list
  add_keywords = list()
  add_keywords = input("Add extra keywords, separated by commas:\n").split(', ')
  #keywords.append(add_keywords)
  keywords.extend(add_keywords)
  print("\nThe updated keyword list is: ")
  for (i, item) in enumerate(keywords, start=0):
    print(i, item)
elif keywords_answer == "no":
  print(f"\nOK, no additions will be made.")

while keywords_answer not in ("yes", "no"):
    keywords_answer = input("Enter yes or no: ")
    if keywords_answer == "yes":
      add_keywords = list()
      add_keywords = input("\nAdd extra keywords, separated by commas: ").split(', ')
      keywords.extend(add_keywords)
      print("\nThe updated keyword list is: ")
      for (i, item) in enumerate(keywords, start=0):
        print(i, item)
    elif keywords_answer == "no":
      print(f"\nOK, no additions will be made.")
      break
    else:
    	print("Please enter yes or no.")


OK, no additions will be made.


In [7]:
keywords_next = input("Do you want to remove keywords from the existing list? ")

if keywords_next == "yes":
  #remove keywords from list
  remove_keywords = list()
  remove_keywords = input("\nEnter the keywords you want removed exactly as they were listed earlier, separated by commas: ").split(', ')

  # looping list of numbers to remove
  for word in remove_keywords:
    # remove it from the list
    while word in keywords:
        keywords.remove(word)

  print("\nThe updated keyword list is: ")
  for (i, item) in enumerate(keywords, start=0):
    print(i, item)

elif keywords_next == "no":
    print(f"\nOK, no removals will be made.")

while keywords_next not in ("yes", "no"):
    keywords_next = input("Enter yes or no: ")

    if keywords_next == "yes":
      #remove keywords from list
      remove_keywords = list()
      remove_keywords = input("\nEnter the keywords you want removed exactly as they were listed earlier, separated by commas: ").split(', ')

      # looping list of numbers to remove
      for word in remove_keywords:
        # remove it from the list
        while word in keywords:
          keywords.remove(word)

      print("\nThe updated keyword list is: ")
      for (i, item) in enumerate(keywords, start=0):
        print(i, item)

    elif keywords_next == "no":
      print(f"\nOK, no removals will be made.")
      break
    else:
    	print("Please enter yes or no.")


OK, no removals will be made.


In [8]:
user_dir = input("\nEnter the directory you want to search exactly as you have it mapped on your computer: ")

answer = input(f"\nIs {user_dir} the correct directory? ")

if answer == "yes":
  # check that the path exists
  if os.path.exists(user_dir):
  # confirm the user_dir search
    print(f"Searching {user_dir} for file contents and compiling data to file, this may take several minutes or several hours depending on size.")
    print("Please keep this window running in the background until complete...")
  else:
    print("Cannot reach network drive. Please check your exact path name and ensure you have permission to access this drive and/or folder./n")
    # repeat user_dir prompt
    retry_dir = input("\nTry again: Enter the directory you want to search exactly as you have it mapped on your computer: ")
    user_dir = retry_dir
    if os.path.exists(user_dir):
    # confirm the user_dir search
      print(f"Searching {user_dir} for file contents and compiling data to file, this may take several minutes or several hours depending on size.")
      print("Please keep this window running in the background until complete...")
    else:
      print("Cannot reach network drive. Exiting now...")
      exit()
elif answer == "no":
  # repeat user_dir prompt
  retry_dir = input("\nTry again: Enter the directory you want to search exactly as you have it mapped on your computer: ")
  user_dir = retry_dir
  if os.path.exists(user_dir):
    # confirm the user_dir search
    print(f"Searching {user_dir} for file contents and compiling data to file, this may take several minutes or several hours depending on size.")
    print("Please keep this window running in the background until complete...")
  else:
      print("Cannot reach ", user_dir, " please confirm exact spelling and/or folder permissions/n")
      # repeat user_dir prompt
      retry_dir = input("\nTry again: Enter the directory you want to search exactly as you have it mapped on your computer: ")
      user_dir = retry_dir
      if os.path.exists(user_dir):
      # confirm the user_dir search
        print(f"Searching {user_dir} for file contents and compiling data to file, this may take several minutes or several hours depending on size.")
        print("Please keep this window running in the background until complete...")
      else:
        print("Cannot reach network drive. Exiting now...")
        exit()
while answer not in ("yes", "no"):
    answer = input("Enter yes or no: ")
    if answer == "yes":
      # confirm the user_dir search
      print(f"Searching {user_dir} for file contents and compiling data to file, this may take several minutes or several hours depending on size. /nPlease keep this window running in the background until complete...")
    elif answer == "no":
      # repeat user_dir prompt
      retry_dir = input("\nTry again: Enter the directory you want to search exactly as you have it mapped on your computer: ")
      user_dir = retry_dir
      if os.path.exists(user_dir):
      # confirm the user_dir search
        print(f"Searching {user_dir} for file contents and compiling data to file, this may take several minutes or several hours depending on size.")
        print("Please keep this window running in the background until complete...")
      else:
        print("Cannot reach network drive. Exiting now...")
        exit()
      break
    else:
    	print("Please enter yes or no.")

Searching P:\Test_Folder_HH for file contents and compiling data to file, this may take several minutes or several hours depending on size.
Please keep this window running in the background until complete...


In [12]:
# search word docs
match_items = []
file_names = os.listdir(user_dir)
docx_names = [file for file in file_names if file.endswith('.docx')]
docx_names = [os.path.join(user_dir, file) for file in docx_names]

for file in docx_names:
    document = Document(file)
    for paragraph in document.paragraphs:
        for kw in keywords:
            regex = re.compile(kw)
            if regex.search(paragraph.text):
            #if kw in paragraph.text:
                match_items.append([file, kw])

searched_df = pd.DataFrame(
    match_items,
    columns=['file_name', 'keyword_match'],
    index=[i[0] for i in match_items]
)

print(searched_df)
searched_df.to_csv(SaveName, index=False)

                                                 file_name  \
P:\Test_Folder_HH\test1.docx  P:\Test_Folder_HH\test1.docx   
P:\Test_Folder_HH\test1.docx  P:\Test_Folder_HH\test1.docx   
P:\Test_Folder_HH\test1.docx  P:\Test_Folder_HH\test1.docx   
P:\Test_Folder_HH\test2.docx  P:\Test_Folder_HH\test2.docx   
P:\Test_Folder_HH\test2.docx  P:\Test_Folder_HH\test2.docx   
P:\Test_Folder_HH\test3.docx  P:\Test_Folder_HH\test3.docx   
P:\Test_Folder_HH\test3.docx  P:\Test_Folder_HH\test3.docx   
P:\Test_Folder_HH\test3.docx  P:\Test_Folder_HH\test3.docx   
P:\Test_Folder_HH\test3.docx  P:\Test_Folder_HH\test3.docx   
P:\Test_Folder_HH\test4.docx  P:\Test_Folder_HH\test4.docx   
P:\Test_Folder_HH\test4.docx  P:\Test_Folder_HH\test4.docx   
P:\Test_Folder_HH\test4.docx  P:\Test_Folder_HH\test4.docx   
P:\Test_Folder_HH\test5.docx  P:\Test_Folder_HH\test5.docx   
P:\Test_Folder_HH\test6.docx  P:\Test_Folder_HH\test6.docx   
P:\Test_Folder_HH\test6.docx  P:\Test_Folder_HH\test6.docx   
P:\Test_

In [12]:
match_pdf_items = []
pdf_names = [file for file in file_names if file.endswith('.pdf')]
pdf_names = [os.path.join(user_dir, file) for file in pdf_names]

for file in pdf_names:
  with open(file, 'rb') as f:
    pdf_text = slate.PDF(f).text()
    print(pdf_text)

abuse 
Traumatic 6a 
Harassment Harass harassment Assault 
ﬁrst aid First Aid First aid ﬁrst Aid 
SIRP 
SIRP 
Bullying sirp 
suicide 
assault abused 


In [19]:
match_pdf_items = []
pdf_names = [file for file in file_names if file.endswith('.pdf')]
pdf_names = [os.path.join(user_dir, file) for file in pdf_names]

for file in pdf_names:
  with open(file, 'rb') as f:
    pdf_text = slate.PDF(f).text()  
    for kw in keywords:
      regex = re.compile(kw)
      if regex.search(pdf_text):
        match_pdf_items.append([file, kw])

searched_pdf = pd.DataFrame(
    match_pdf_items,
    columns=['file_name', 'keyword_match'],
    index=[i[0] for i in match_pdf_items]
)

print(searched_pdf)
searched_pdf.to_csv(SaveName, mode='a', index=False, header=False)

                                                     file_name  \
P:\Test_Folder_HH\PDFtest1.pdf  P:\Test_Folder_HH\PDFtest1.pdf   
P:\Test_Folder_HH\PDFtest2.pdf  P:\Test_Folder_HH\PDFtest2.pdf   
P:\Test_Folder_HH\PDFtest2.pdf  P:\Test_Folder_HH\PDFtest2.pdf   
P:\Test_Folder_HH\PDFtest3.pdf  P:\Test_Folder_HH\PDFtest3.pdf   
P:\Test_Folder_HH\PDFtest3.pdf  P:\Test_Folder_HH\PDFtest3.pdf   
P:\Test_Folder_HH\PDFtest5.pdf  P:\Test_Folder_HH\PDFtest5.pdf   
P:\Test_Folder_HH\PDFtest6.pdf  P:\Test_Folder_HH\PDFtest6.pdf   
P:\Test_Folder_HH\PDFtest7.pdf  P:\Test_Folder_HH\PDFtest7.pdf   
P:\Test_Folder_HH\PDFtest7.pdf  P:\Test_Folder_HH\PDFtest7.pdf   
P:\Test_Folder_HH\PDFtest8.pdf  P:\Test_Folder_HH\PDFtest8.pdf   
P:\Test_Folder_HH\PDFtest9.pdf  P:\Test_Folder_HH\PDFtest9.pdf   
P:\Test_Folder_HH\PDFtest9.pdf  P:\Test_Folder_HH\PDFtest9.pdf   

                                      keyword_match  
P:\Test_Folder_HH\PDFtest1.pdf        \babus|\bAbus  
P:\Test_Folder_HH\PDFtest2.pdf   

In [33]:
match_excel_items = []
excel_names = [file for file in file_names if file.endswith('.xlsx')]
excel_names = [os.path.join(user_dir, file) for file in excel_names]

for file in excel_names:
  wb = openpyxl.load_workbook(file)
  for ws in wb.worksheets:
    for row in ws.iter_rows():
      for cell in row:
        for kw in keywords:
          regex = re.compile(kw)
          if regex.search(cell.value):
            match_excel_items.append([file, kw])

searched_excel = pd.DataFrame(
    match_excel_items,
    columns=['file_name', 'keyword_match'],
    index=[i[0] for i in match_excel_items]
)

print(searched_excel)
searched_excel.to_csv(SaveName, mode='a', index=False, header=False)

                                                               file_name  \
P:\Test_Folder_HH\Excel_Test1.xlsx    P:\Test_Folder_HH\Excel_Test1.xlsx   
P:\Test_Folder_HH\Excel_Test10.xlsx  P:\Test_Folder_HH\Excel_Test10.xlsx   
P:\Test_Folder_HH\Excel_Test10.xlsx  P:\Test_Folder_HH\Excel_Test10.xlsx   
P:\Test_Folder_HH\Excel_Test2.xlsx    P:\Test_Folder_HH\Excel_Test2.xlsx   
P:\Test_Folder_HH\Excel_Test3.xlsx    P:\Test_Folder_HH\Excel_Test3.xlsx   
P:\Test_Folder_HH\Excel_Test3.xlsx    P:\Test_Folder_HH\Excel_Test3.xlsx   
P:\Test_Folder_HH\Excel_Test4.xlsx    P:\Test_Folder_HH\Excel_Test4.xlsx   
P:\Test_Folder_HH\Excel_Test4.xlsx    P:\Test_Folder_HH\Excel_Test4.xlsx   
P:\Test_Folder_HH\Excel_Test5.xlsx    P:\Test_Folder_HH\Excel_Test5.xlsx   
P:\Test_Folder_HH\Excel_Test5.xlsx    P:\Test_Folder_HH\Excel_Test5.xlsx   
P:\Test_Folder_HH\Excel_Test6.xlsx    P:\Test_Folder_HH\Excel_Test6.xlsx   
P:\Test_Folder_HH\Excel_Test6.xlsx    P:\Test_Folder_HH\Excel_Test6.xlsx   
P:\Test_Fold

In [36]:
match_csv_items = []
csv_names = [file for file in file_names if file.endswith('.csv')]
csv_names = [os.path.join(user_dir, file) for file in csv_names]

for file in csv_names:
    with open(file, 'rt') as f:
        reader = csv.reader(f)
        for row in reader:
            for field in row:
                for kw in keywords:
                    regex = re.compile(kw)
                    if regex.search(field):
                        match_csv_items.append([file, kw])

searched_csv = pd.DataFrame(
    match_csv_items,
    columns=['file_name', 'keyword_match'],
    index=[i[0] for i in match_csv_items]
)

print(searched_csv)
searched_csv.to_csv(SaveName, mode='a', index=False, header=False)

                                                         file_name  \
P:\Test_Folder_HH\CSV_Test1.csv    P:\Test_Folder_HH\CSV_Test1.csv   
P:\Test_Folder_HH\CSV_Test10.csv  P:\Test_Folder_HH\CSV_Test10.csv   
P:\Test_Folder_HH\CSV_Test10.csv  P:\Test_Folder_HH\CSV_Test10.csv   
P:\Test_Folder_HH\CSV_Test2.csv    P:\Test_Folder_HH\CSV_Test2.csv   
P:\Test_Folder_HH\CSV_Test3.csv    P:\Test_Folder_HH\CSV_Test3.csv   
P:\Test_Folder_HH\CSV_Test4.csv    P:\Test_Folder_HH\CSV_Test4.csv   
P:\Test_Folder_HH\CSV_Test4.csv    P:\Test_Folder_HH\CSV_Test4.csv   
P:\Test_Folder_HH\CSV_Test5.csv    P:\Test_Folder_HH\CSV_Test5.csv   
P:\Test_Folder_HH\CSV_Test5.csv    P:\Test_Folder_HH\CSV_Test5.csv   
P:\Test_Folder_HH\CSV_Test6.csv    P:\Test_Folder_HH\CSV_Test6.csv   
P:\Test_Folder_HH\CSV_Test6.csv    P:\Test_Folder_HH\CSV_Test6.csv   
P:\Test_Folder_HH\CSV_Test7.csv    P:\Test_Folder_HH\CSV_Test7.csv   
P:\Test_Folder_HH\CSV_Test7.csv    P:\Test_Folder_HH\CSV_Test7.csv   
P:\Test_Folder_HH\CS

In [37]:
match_text_items = []
text_names = [file for file in file_names if file.endswith('.txt')]
text_names = [os.path.join(user_dir, file) for file in text_names]

for file in text_names:
    with open(file, 'r') as f:
        content = f.read()
        for kw in keywords:
            regex = re.compile(kw)
            if regex.search(content):
                match_text_items.append([file, kw])

searched_text = pd.DataFrame(
    match_text_items,
    columns=['file_name', 'keyword_match'],
    index=[i[0] for i in match_text_items]
)

print(searched_text)
searched_text.to_csv(SaveName, mode='a', index=False, header=False)

                                                         file_name  \
P:\Test_Folder_HH\Text_Test1.txt  P:\Test_Folder_HH\Text_Test1.txt   
P:\Test_Folder_HH\Text_Test2.txt  P:\Test_Folder_HH\Text_Test2.txt   
P:\Test_Folder_HH\Text_Test2.txt  P:\Test_Folder_HH\Text_Test2.txt   
P:\Test_Folder_HH\Text_Test3.txt  P:\Test_Folder_HH\Text_Test3.txt   
P:\Test_Folder_HH\Text_Test3.txt  P:\Test_Folder_HH\Text_Test3.txt   
P:\Test_Folder_HH\Text_Test4.txt  P:\Test_Folder_HH\Text_Test4.txt   
P:\Test_Folder_HH\Text_Test4.txt  P:\Test_Folder_HH\Text_Test4.txt   
P:\Test_Folder_HH\Text_Test5.txt  P:\Test_Folder_HH\Text_Test5.txt   
P:\Test_Folder_HH\Text_Test6.txt  P:\Test_Folder_HH\Text_Test6.txt   
P:\Test_Folder_HH\Text_Test7.txt  P:\Test_Folder_HH\Text_Test7.txt   
P:\Test_Folder_HH\Text_Test8.txt  P:\Test_Folder_HH\Text_Test8.txt   
P:\Test_Folder_HH\Text_Test8.txt  P:\Test_Folder_HH\Text_Test8.txt   
P:\Test_Folder_HH\Text_Test9.txt  P:\Test_Folder_HH\Text_Test9.txt   

                   